# Booru scraper

In [ ]:
import json
import pandas as pd
import time
import requests
import numpy as np
import sqlite3

In [ ]:
connection = sqlite3.connect("booru.db")


In [ ]:
limit = 1000
# get url for user and page


def get_url(user, page):
    return (
        f"https://gelbooru.com/index.php?page=dapi&q=index&json=1&limit={limit}&"
        + f"s=post&tags=fav:{user}&pid={page}&"
        + "api_key=c3ca8d6069fba65a4fd6e81e412712c6607215b3f4710e62f6ee3de44a8350df&user_id=347815"
    )


max_id = connection.execute("SELECT MAX(user_id) FROM users").fetchone()[0]
id_to_start = max_id if max_id >= 1 else 1
for user_id in range(id_to_start, 1_000_000):
    print(f"User: {user_id}")
    page = 0
    user_favs = pd.DataFrame()
    print("Page: ", end="")
    while True:
        print(f"{page}; ", end="")
        url = get_url(user_id, page)
        response = requests.request("GET", url)
        try:
            response = response.json()
        except json.decoder.JSONDecodeError as e:
            print(response.text)
            break
        if "post" not in response:
            break
        user_favs = user_favs.append(pd.DataFrame(response["post"]))
        count = response["@attributes"]["count"]
        page += 1
        if page * limit > count:
            break
    print("")
    if len(user_favs) == 0:
        continue

    connection.execute("INSERT OR IGNORE INTO users (user_id) VALUES (?)", (user_id,))

    save_start_time = time.time_ns() // 1000000
    user_favs.replace(
        {"True": True, "False": False, "true": True, "false": False}, inplace=True
    )
    connection.executemany(
        "INSERT OR IGNORE INTO posts (id, created_at, score, width, height, md5, directory, image, rating, source, change, owner, creator_id, parent_id, sample, preview_height, preview_width, tags, title, has_notes, has_comments, file_url, preview_url, sample_url, sample_height, sample_width, status, post_locked, has_children) "
        + "VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
        user_favs.values.tolist(),
    )

    user_favs["user_id"] = user_id
    connection.executemany(
        "INSERT OR IGNORE INTO favs (user_id, post_id) VALUES (?, ?)",
        user_favs[["user_id", "id"]].rename(columns={"id": "post_id"}).values.tolist(),
    )
    connection.commit()

    save_duration = time.time_ns() // 1000000 - save_start_time
    print(f"Saved in {save_duration} ms")
